## RUN-3

- using batch normalization after each and every activation function.
   - add batch normalization function in model.py file
- scale the data before feeding into the data.
   - change the data_generator function for doing this
- decrease the learning rate from 0.1 to 0.01 and increase the epochs to 40
- we created a validation generator to look more like data generator
   - This will allow us to interpret the output appropriately
- We have 708 batches approx , with 196 images per batch - total is 138768 images per epoch 
- Reduce the number of transformations you are doing 
- During the testing, rotate and flip the Images 10 times each and take the average of the classes.

In [1]:
from helper import *
from model import *
import pandas as pd
import numpy as np 
import cv2
import glob
from sklearn.cross_validation import train_test_split

import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [2]:
labels = pd.read_table("/data/dr/trainLabels.csv",sep=",",index_col=["image"])
labels = labels.drop("492_right")

In [3]:
filelist = glob.glob("/data/dr/data/sample_270_270/*.jpeg")

x_train,x_valid,y_train, y_valid = train_test_split(filelist,labels,
  test_size = 0.10, random_state = 20)



## train_test split 
x_train,x_test,y_train, y_test = train_test_split(x_train,y_train,
  test_size = 0.10, random_state = 20)

In [4]:
model_train = model((270,270,3))

In [5]:
model_train.fit_generator(data_generator(y_train), samples_per_epoch=95200,
                          nb_epoch=40, verbose=1, callbacks=[], validation_data=valid_generator(y_valid), 
                          class_weight=None, max_q_size=10, nb_worker=1)

Epoch 1/40
95200/95200 [==============================] - 937s - loss: 1.2362 - acc: 0.4861 - val_loss: 1.6015 - val_acc: 0.3760
Epoch 2/40
95200/95200 [==============================] - 930s - loss: 1.1407 - acc: 0.5412 - val_loss: 1.1046 - val_acc: 0.5702
Epoch 3/40
95200/95200 [==============================] - 918s - loss: 1.0729 - acc: 0.5669 - val_loss: 1.3162 - val_acc: 0.4669
Epoch 4/40
95200/95200 [==============================] - 920s - loss: 1.0059 - acc: 0.5913 - val_loss: 1.3874 - val_acc: 0.5124
Epoch 5/40
95200/95200 [==============================] - 944s - loss: 0.9863 - acc: 0.5951 - val_loss: 1.4586 - val_acc: 0.5041
Epoch 6/40
95200/95200 [==============================] - 964s - loss: 0.9416 - acc: 0.6158 - val_loss: 3.1889 - val_acc: 0.4091
Epoch 7/40
95200/95200 [==============================] - 914s - loss: 0.9037 - acc: 0.6300 - val_loss: 1.5177 - val_acc: 0.5331
Epoch 8/40
95200/95200 [==============================] - 905s - loss: 0.8893 - acc: 0.6378 - val

In [6]:
model_train.save("run3.h")
model_train.save_weights("run3_weights.h5")

In [39]:
# simple Testing without using any transformtions on the test data 
x_test = np.array([cv2.imread("/data/dr/data/sample_270_270/"+i+".jpeg") for i in y_test.index])
x_test = x_test/np.amax(x_test)
classes = model_train.predict_classes(x_test, batch_size=8)

3160/3162 [============================>.] - ETA: 0s

In [40]:
from sklearn import metrics
kappa_score = metrics.cohen_kappa_score( np.array(y_test['level']), classes)
print (kappa_score)

0.0673708409705


In [9]:
np.unique(classes, return_counts=True)

(array([0, 2]), array([3161,    1]))

### testing on Individual Image doing all transformation and taking vote accordingly.

- Each Image need to be rotated in all the angles starting from [30 to 360]
- each rotated Image must be flipped both horizantally and vertically.
- Predict the class of each Image and take the maximum voting.

In [60]:
def test_Image_generator(dataframe,location = "/data/dr/data/sample_270_270/"):
    image_label= []
    index = dataframe.index
    for i in range(len(index)):
        image = np.array(cv2.imread(location+index[i]+".jpeg"))
        i_rotate = np.array([rotate(image,i) for i in [30,60,90,135,150,180,225,270,315]])
        i_flip =np.array([cv2.flip(i_rotate[j],i) for j in range(len(i_rotate)) for i in [0,1]])
        image= image[np.newaxis,:,:,:]
        X_test = np.concatenate((image,i_rotate,i_flip))
        X_test = X_test/np.amax(X_test) #normalize data 
        classes = model_train.predict_classes(X_test,batch_size=7,verbose =0)
        classes = list(classes)
        x = np.bincount(classes).argmax()
        image_label.append(x)
        print(x)
    return image_label

In [61]:
test_label = test_Image_generator(y_test)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
3
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0


In [64]:
np.unique(test_label, return_counts=True)

(array([0, 1, 2, 3]), array([3027,    7,   81,   47]))

In [63]:
from sklearn import metrics
kappa_score = metrics.cohen_kappa_score( np.array(y_test['level']), test_label)
print (kappa_score)

0.0914010942906
